In [56]:
import numpy as np
from scipy import stats

We define a function to generate a confidence interval given a vector of sampled values

In [57]:
def getCI(vals):
    n = len(vals)
    CL = 0.95 # confidence level
    DF = n-1 # degrees of freedom
    z = np.abs(stats.t.ppf((1-CL)/2,DF))
    mean = np.mean(vals)
    std = np.std(vals, ddof = 1)
    u = mean + z*std/np.sqrt(n)
    l = mean - z*std/np.sqrt(n)
    return mean, l, u

<h3> (1) Estimate integral using crude Monte Carlo </h3>

We sample 100 values from the uniform distribution and apply the crude method to get the estimate of the integral.

In [58]:
us = np.random.uniform(0,1, size = 100)
exp = np.exp(us)
print('The variance of the estimation is: {:.4f}'.format(np.var(exp)))
mean, l, u = getCI(exp)
print('The estimate of the integral is: {:.4f}'.format(mean))
print('With the following confidence interval: {:.4f}, {:.4f}'.format(l,u))
print('Analytical solution of the integral: {:.4f}'.format(np.e - 1))

The variance of the estimation is: 0.2261
The estimate of the integral is: 1.6194
With the following confidence interval: 1.5246, 1.7143
Analytical solution of the integral: 1.7183


<h3> (2) Estimate integral using antithetic variables </h3>

We sample 100 values from the uniform distribution and we estimate the integral using antithetic variables

In [59]:
us = np.random.uniform(0,1, size = 100)
exp = np.exp(us)
y = (exp + np.e/exp)/2

print('The variance of the estimation is: {:.4f}'.format(np.var(y)))
mean, l, u = getCI(y)
print('The estimate of the integral is: {:.4f}'.format(mean))
print('With the following confidence interval: {:.4f}, {:.4f}'.format(l,u))
print('Analytical solution of the integral: {:.4f}'.format(np.e - 1))

The variance of the estimation is: 0.0040
The estimate of the integral is: 1.7177
With the following confidence interval: 1.7051, 1.7303
Analytical solution of the integral: 1.7183


Note that the variance is reduced a lot with respect to the crude method

<h3> (3) Estimate integral using control variates </h3>

In [60]:
us = np.random.uniform(0,1, size = 100)
exp = np.exp(us)
c = -np.cov(exp, us)[0,1]/np.var(us)
z = exp + c*(us - 1/2)
print('The variance of the estimation is: {:.4f}'.format(np.var(z)))
mean, l, u = getCI(z)
print('The estimate of the integral is: {:.4f}'.format(mean))
print('With the following confidence interval: {:.4f}, {:.4f}'.format(l,u))
print('Analytical solution of the integral: {:.4f}'.format(np.e - 1))

The variance of the estimation is: 0.0041
The estimate of the integral is: 1.7233
With the following confidence interval: 1.7106, 1.7360
Analytical solution of the integral: 1.7183


Also in this case the variance is reduced with respect the crude method.

<h3> (4) Estimate integral using stratified sampling </h3>

We use stratified sampling with 10 strata to get the estimate of the integral. Using this method we are able to reduce the variance of the estimation by another order of magnitude

In [61]:
w = [] # The final list will contain ten values for the estimation of the integral
for i in range(10):
    us = np.random.uniform(0,1,10) # Each sample is based on ten uniformly distributed values
    w.append(np.sum([np.exp((j + us[j])/10) for j in range(10)])/10) # Stratified sampling

print('The variance of the estimation is: {:.4f}'.format(np.var(w)))
mean, l, u = getCI(w)
print('The estimate of the integral is: {:.4f}'.format(mean))
print('With the following confidence interval: {:.4f}, {:.4f}'.format(l,u))
print('Analytical solution of the integral: {:.4f}'.format(np.e - 1))

The variance of the estimation is: 0.0002
The estimate of the integral is: 1.7141
With the following confidence interval: 1.7039, 1.7242
Analytical solution of the integral: 1.7183


We can note that in all the cases we got quite accurate estimations of the integral, but with different estimations of the variance. In detail, the crude Monte Carlo estimation has the most variance, which is then reduced in the other methods. This is reflected also by the width of the confidence intervals.

<h3> (5) Control variates for blocking queueing system simulation </h3>

We use the control variate method to reduce the variance in the estimation of the blocked fraction of customers in the queueing system problem. As control variate, we use the mean arrival time in the simulation.

The following function simulates the problem using a Poisson process for the arrivals and exponential service times. It returns the fraction of blocked customers and the mean arrival time.

In [82]:
def simulate_queue(nserver, customers, mean_st, mean_tbc):
    server_time = np.zeros(nserver)
    time = 0
    blocked = 0
    t_arrival_time = 0
    for _ in range(customers):
        delta_arrival_time = stats.expon.rvs(scale = mean_tbc, size = 1)[0]
        t_arrival_time += delta_arrival_time
        time += delta_arrival_time
        min_server = np.min(server_time)
        idx_min_server = np.argmin(server_time)
        if time < min_server:
            blocked +=1
        else:
            delta_service_time = stats.expon.rvs(scale = mean_st, size = 1)
            server_time[idx_min_server] = time + delta_service_time
    
    return blocked/customers, t_arrival_time/customers

We initialize some parameters of interest

In [84]:
nserver = 10 # number of service units
mean_st = 8 # mean service time
mean_tbc = 1 # mean time between customers
customers = 10000 #number of customers for each simulation
nsim = 10 # number of simulations

Then we perform ten simulations, saving for each one the fraction of blocked customers and the mean arrival time

In [101]:
runs = [] # this list will contain the fraction of blocked customers for each simulation
arrivals = [] # this list will contain the mean arrival time for each simulation
for i in range(nsim):
    blocked, arrival = simulate_queue(nserver, customers, mean_st, mean_tbc)
    runs.append(blocked)
    arrivals.append(arrival)

We use the control variates method to reduce the variance in the estimation

In [102]:
runs = np.array(runs)
arrivals = np.array(arrivals)

c = -np.cov(runs, arrivals)[0,1]/np.var(arrivals)

z = runs + c*(arrivals - mean_tbc)

mean_r, l_r, u_r = getCI(runs)
mean_z, l_z, u_z = getCI(z)

print('The estimate of the fraction of blocked customers (without variance reduction) is: {:.4f}'.format(mean_r))
print('With the following confidence interval: {:.4f}, {:.4f}\n'.format(l_r,u_r))

print('The estimate of the fraction of blocked customers (with variance reduction) is: {:.4f}'.format(mean_z))
print('With the following confidence interval: {:.4f}, {:.4f}'.format(l_z,u_z))

print('The variance without control variates is:',np.var(runs))
print('The variance with control variates is:',np.var(z))

The estimate of the fraction of blocked customers (without variance reduction) is: 0.1208
With the following confidence interval: 0.1158, 0.1257

The estimate of the fraction of blocked customers (with variance reduction) is: 0.1196
With the following confidence interval: 0.1168, 0.1224
The variance without control variates is: 4.362840000000002e-05
The variance with control variates is: 1.4024515859755557e-05


We can note that the variance has been reduced using control variates

<h3> (6) Common random numbers in queueing system simulation </h3>

We define two functions able to sample from the exponential and hyperexponential distributions given some samples drawn from the uniform distribution passed as parameters.

In [ ]:
def getExp(lam, us):
    exp = -np.log(us)/lam
    return exp 

def getHyperExp(p, lam1, lam2, u1, u2):
    res = np.zeros(len(u1))
    res[u2 <= p] = getExp(lam = lam1, us = u1[u2 <=p])
    res[u2 > p] = getExp(lam = lam2, us = u1[u2 > p])
    return res

For the Common Random Numbers method to work, we need to ensure that the two systems (one having exponential inter-arrival times, the other hyperexponential), run on the same sequence of random numbers, uniformly distributed between 0 and 1. To do so, we let the caller of the function set a seed that is then used to sample the sequence of values used to obtain the arrival times. The aim is to call the function in both types 'Exp' (corresponding to exponential arrival times) and 'Hyp' (hyperexponential arrival times) with the same seed, so we can fairly compare the results.

In [103]:
def simulate_queue_q2(nserver, customers, mean_st, mean_tbc, type = 'Exp', seed = 0):
    np.random.seed(seed)
    u1 = np.random.uniform(0,1, customers)
    u2 = np.random.uniform(0,1, customers)
    server_time = np.zeros(nserver)
    time = 0
    blocked = 0
    if type == 'Exp':
        arrival_times = getExp(lam = mean_tbc, us = u1)
    elif type == 'Hyp':
        arrival_times = getHyperExp(0.8, 0.8333, 5, u1, u2)
    for i in range(customers):
        delta_arrival_time = arrival_times[i]
        time += delta_arrival_time
        min_server = np.min(server_time)
        idx_min_server = np.argmin(server_time)
        if time < min_server:
            blocked += 1
        else:
            server_time[idx_min_server] = time + stats.expon.rvs(scale = mean_st, size = 1)
    
    return blocked/customers

We initialize the parameters

In [104]:
nserver = 10
mean_st = 8
mean_tbc = 1
customers = 10000
nsim = 10

We perform ten simulations of the two processes using ten different seeds, saving the blocked fraction of customers for each run. Then we perform a paired t-test to see if there is a difference between the two processes.

In [107]:
runs = []
for i in range(10):
    runs.append([simulate_queue_q2(nserver, customers, mean_st, mean_tbc, 'Hyp', i), simulate_queue_q2(nserver, customers, mean_st, mean_tbc, 'Exp', i)])

runs = np.array(runs)

print(stats.ttest_rel(runs[:,0], runs[:,1]))
print('Estimated difference between the two processes: {:.4f}'.format(np.mean(runs[:,0] - runs[:,1])))

Ttest_relResult(statistic=16.68412262034125, pvalue=4.462657602263569e-08)
Estimated difference between the two processes: 0.0191


The p-value is very low, indicating a strong suggestion that the two processes perform differently. The estimation of the difference in performance is almost 2%, as printed above

<h3> (7) Monte Carlo on standard normal random variable </h3>

First, we use the crude Monte Carlo method to estimated the desired probability, sampling 10000 values from the standard normal distribution and seeing how many are above the threshold. This will give a rough estimate of the probability. 

In [65]:
a = 4
tot = 10000
values = np.random.randn(tot) > a

mean, l, u = getCI(values)
print('The estimate of the probability is: {:.6f}'.format(mean))
print('With the following confidence interval: {:.6f}, {:.6f}'.format(l,u))
print('The true value for the probability is: {:.6f}'.format(1 - stats.norm.cdf(a)))

The estimate of the probability is: 0.000100
With the following confidence interval: -0.000096, 0.000296
The true value for the probability is: 0.000032


Using a=2 we get somewhat accurate estimation of the probability, while for a=4 the result is much worse, because we are estimating the probability of a more extreme outcome and crude Monte Carlo struggles to estimate it with few samples. We would therefore need more samples to estimate it correctly.

The problem can be solved using Importance Sampling as follows, always using 10000 samples.

In [66]:
a = 4
s = 1
tot = 10000

samples = stats.norm.rvs(loc = a, scale = s, size = tot)

h = samples > a
f = stats.norm.pdf(samples)
g = stats.norm.pdf(samples, loc = a, scale = s)

Z = h * f / g

mean, l, u = getCI(Z)
print('The estimate of the probability is: {:.6f}'.format(mean))
print('With the following confidence interval: {:.6f}, {:.6f}'.format(l,u))
print('The true value for the probability is: {:.6f}'.format(1 - stats.norm.cdf(a)))

The estimate of the probability is: 0.000033
With the following confidence interval: 0.000031, 0.000034
The true value for the probability is: 0.000032


<h3> (8) Exponential importance sampling </h3>

We include a photo for the derivation of the value of λ which minimizes the required variance

![picture 1](../images/cce642c549bddefeb7e43662c6ea703da81334325ef94f3f5ae6f03b234186fb.png)  


We verify the result using simulation, using the value of λ found analytically

In [67]:
lam = 1.35483
size = 100000

values = stats.expon.rvs(scale = 1/lam, size = size) # we get values from the sample distribution g

f = np.logical_and(values <= 1, values>=0)
h = np.exp(values)
g = lam*np.exp(-lam*values)

res = f * h / g

mean, l, u = getCI(res)
print('The estimate of the integral is: {:.4f}'.format(mean))
print('With the following confidence interval: {:.4f}, {:.4f}'.format(l,u))
print('Analytical solution of the integral: {:.4f}'.format(np.e - 1))
print('The estimated variance is the following: {:.4f}'.format(np.var(res, ddof = 1)))

The estimate of the integral is: 1.7278
With the following confidence interval: 1.7168, 1.7388
Analytical solution of the integral: 1.7183
The estimated variance is the following: 3.1552


We can see that we have a good estimation of the integral, but with a large variance with respect to the method used in the first exercises of this notebook.

<h3> (9) Pareto IS estimator </h3>

We perform importance sampling for the estimation of the mean for the Pareto distribution. The sampling distribution is the first moment distribution: in the Pareto case, this is distributed again as a Pareto with the k parameter decreased by 1. Note that we use β = 1.

In [116]:
k = 1.05
size = 10000
values = stats.pareto.rvs(k-1, size = size) # getting values from the sampling distribution

h = values
f = stats.pareto.pdf(values, k)
g = stats.pareto.pdf(values, k-1)

res = h * f / g

mean, l, u = getCI(res)
print(f'The estimate of the mean is: {mean}')
print(f'With the following confidence interval: {l},{u}')
print(f'Analytical value of the mean: {k/(k-1)}')

The estimate of the mean is: 21.000000000000075
With the following confidence interval: 21.00000000000007,21.000000000000078
Analytical value of the mean: 20.999999999999982


Using importance sampling in this case is very helpful since it allows to estimate the mean with very high precision: this task is very difficult with the simple estimation, as seen in exercise 3.